In [54]:
import json
import pandas as pd
import numpy as np
import re
from pyvi import ViTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

In [55]:
with open('data/dataset.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)

In [56]:
tmp_label = ['Apps/Games', 'Internet', 'Khám phá', 'Mobile', 'Tin ICT', 'Đồ chơi số']
df = df[df['category'].isin(tmp_label)]
df['text'] = df['title'] + ". " + df['subtitle']
df = df[['text', 'category']]
print(df.head())

                                                text    category
0  Cá nhân hóa trải nghiệm bằng công nghệ tiên ti...  Apps/Games
1  Microsoft ra mắt Office 2024. Office 2024 đã c...  Apps/Games
2  Những thành tựu khó đạt được nhất trong Black ...  Apps/Games
3  Tựa game "đắt" nhất trên Steam, mua đủ hết nội...  Apps/Games
4  CEO Game Science trả lời hài hước về khả năng ...  Apps/Games


In [57]:
def read_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = [line.strip() for line in file if line.strip()]
    return lines

stopwords = read_txt('stopwords.txt')
print(len(stopwords))

1943


In [58]:
def read_txt_to_dict(file_path):
    dictionary = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if line.strip():  # Bỏ qua các dòng trống
                # Tách dòng theo khoảng trắng hoặc tab, chỉ tách lần đầu tiên để có cặp key-value
                key, value = line.strip().split(maxsplit=1)
                dictionary[key] = value
    return dictionary

acronyms = read_txt_to_dict('acronyms_vi.txt')
print(acronyms)

{'ship': 'vận chuyển', 'shop': 'cửa hàng', 'm': 'mình', 'mik': 'mình', 'ko': 'không', 'k': 'không', 'kh': 'không', 'khong': 'không', 'kg': 'không', 'khg': 'không', 'tl': 'trả lời', 'rep': 'trả lời', 'r': 'rồi', 'fb': 'facebook', 'face': 'faceook', 'thanks': 'cảm ơn', 'thank': 'cảm ơn', 'tks': 'cảm ơn', 'tk': 'cảm ơn', 'ok': 'tốt', 'oki': 'tốt', 'okie': 'tốt', 'sp': 'sản phẩm', 'dc': 'được', 'vs': 'với', 'đt': 'điện thoại', 'thjk': 'thích', 'thik': 'thích', 'qá': 'quá', 'trể': 'trễ', 'bgjo': 'bao giờ', 'h': 'giờ', 'qa': 'quá', 'dep': 'đẹp', 'xau': 'xấu', 'ib': 'nhắn tin', 'cute': 'dễ thương', 'sz': 'size', 'good': 'tốt', 'god': 'tốt', 'bt': 'bình thường', 'sd': 'sử dụng', 'ios': 'hệ điều hành', 'vk': 'vợ', 'cung': 'cũng', 'oke': 'tốt', 'thk': 'cảm ơn', 'thnk': 'cảm ơn', 'series': 'bộ', 'ng': 'người', 'onl': 'online', 'ip': 'điện thoại', 'vsoi': 'với', 'trâu': 'tốt', 'edit': 'chỉnh sửa', 'xò': 'tốt', '5sao': 'tốt', 'thơm': 'tốt', 'chs': 'chả hiểu sao', 'klq': 'không liên quan', 'clgt': '

In [59]:
def segmentation(text):
    return ViTokenizer.tokenize(text)

def split_words(text):
    try:
        segmented_text = segmentation(text)
        SPECIAL_CHARACTER = '.,!?;:"\'[]{}()<>@#$%^&*+-/\\|=~`_'
        words = [x.strip(SPECIAL_CHARACTER).lower() for x in segmented_text.split()]
        return " ".join(words)
    except TypeError:
        return ""
    
def replace_acronyms(text, acronyms):
    list_text = text.split(" ")
    for i in range(len(list_text)):
        # Kiểm tra nếu từ hiện tại là một từ viết tắt trong acronyms (dict)
        if list_text[i] in acronyms:
            # Thay thế từ viết tắt bằng từ đầy đủ
            list_text[i] = acronyms[list_text[i]]
    return " ".join(list_text)

In [60]:
def clean_text(text):
    # Chuyển văn bản về chữ thường
    text = text.lower()
    
    text = replace_acronyms(text, acronyms)
    
    text = split_words(text)
    
    # Loại bỏ ký tự đặc biệt và dấu câu
    text = re.sub(r'\W', ' ', text)
    
    # Loại bỏ các số
    text = re.sub(r'\d+', '', text)
    
    # Loại bỏ các từ dừng (stopwords)
    text = ' '.join([word for word in text.split() if word not in stopwords])
    
    return text

In [61]:
df['text'] = df['text'].apply(clean_text)
print(df.head())

                                                text    category
0  cá_nhân_hóa trải nghiệm công_nghệ tiên_tiến ng...  Apps/Games
1  microsoft ra_mắt office office chính_thức ra_m...  Apps/Games
2  thành_tựu black myth wukong youtube ải như_khô...  Apps/Games
3  tựa trò_chơi đắt steam mua nội_dung tốn triệu ...  Apps/Games
4  ceo trò_chơi science trả_lời hài_hước khả_năng...  Apps/Games


In [65]:
df['text'][5643]

'tự_sự kỹ_sư trẻ xóa ví tiền điện_tử chứa bitcoin cảm_thấy nhàm_chán tiền điện_tử khai_thác máy_tính cá_nhân xử_lý amd phenom x ram mb'